In [16]:
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import numpy as np
import json
import string
import re
import time
from collections import defaultdict

from array import array
import math
import collections
from numpy import linalg as la

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/iliasasskali/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
# Read json and store the text of each tweet into a list
def read_json(docs_path):
    with open(docs_path) as fp:
        lines = json.load(fp)

    print("Total number of tweets in the dataset: {}".format(len(tweets)))
    return lines

docs_path = 'inputs/dataset_tweets_WHO.txt'
lines = read_json(docs_path)

tweets = []
for tweetId in lines:
    tweets.append(lines[str(tweetId)]["full_text"])

Total number of tweets in the dataset: 2399


In [9]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)

url_pattern = re.compile(r'https?://\S+|www\.\S+')

def remove_urls(line):
    return url_pattern.sub(r'', line)

def remove_emojis(line):
    return emoji_pattern.sub(r'', line)

def build_terms(line):
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    line = line.encode("ascii", "ignore") # Remove unicode characters
    line = line.decode()
    line = line.lower() ## Transform to lowercase
    line = remove_emojis(line) ## Remove emojis, before tokenizing to delete emojis not separated by space with a word
    line = remove_urls(line) ## Remove urls
    line = line.split() ## Tokenize the text to get a list of terms
    line = [w for w in line if w not in stop_words]  ## eliminate the stopwords
    line = [w for w in line if w[0]!='&' and w[-1]!=';'] ## Remove HTML symbol entity codes
    line = [w.strip(string.punctuation.replace('#', '').replace('@', '')) for w in line] ## Remove punctuation except # and @
    line = [stemmer.stem(w) for w in line if w!=''] ## perform stemming and remove empty words
    return line

In [10]:
# Execute to see first 10 original and processed tweets
for tweet in tweets[:10]:
    print("Original tweet:\n" + tweet + "\n")
    print("Processed tweet:\n" + str(build_terms(tweet)) + "\n")

Original tweet:
It's International Day for Disaster Risk Reduction

#OpenWHO has launched a multi-tiered core curriculum to help equip you with the competencies needed to work within public health emergency response.

Start learning today &amp; be #Ready4Response:
👉 https://t.co/hBFFOF0xKL https://t.co/fgZY22RWuS

Processed tweet:
['intern', 'day', 'disast', 'risk', 'reduct', '#openwho', 'launch', 'multi-ti', 'core', 'curriculum', 'help', 'equip', 'compet', 'need', 'work', 'within', 'public', 'health', 'emerg', 'respons', 'start', 'learn', 'today', '#ready4respons']

Original tweet:
#COVID19 has shown how health emergencies and disasters affect entire communities – especially those with weak health systems, and vulnerable populations like migrants, indigenous peoples, and those living in fragile humanitarian conditions. https://t.co/jpUQpnu0V1

Processed tweet:
['#covid19', 'shown', 'health', 'emerg', 'disast', 'affect', 'entir', 'commun', 'especi', 'weak', 'health', 'system', 'vulner'

PART 2: INDEXING AND EVALUATION

In [48]:
# 1. Inverted index:
def find_occurrences(term, tweet_text):
    """
    Argument:
    term -- term to find 
    text -- original where to find terms

    Returns:
    list of positions where the term occurs in the text
    """

def create_index(lines):
    """
    Implement the inverted index
    
    Argument:
    lines -- collection of json tweets
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of documents where these keys appears in (and the positions) as values.
    """
    index = defaultdict(list)
    #title_index = {}  # dictionary to map page titles to page ids
    for tweetId in lines:  # lines contain all tweets, whith the id as key
        tweet_text = lines[str(tweetId)]["full_text"]
        terms = build_terms(tweet_text)
        #title = line_arr[1]
        #title_index[page_id]=title  ## we do not need to apply get terms to title because it used only to print titles and not in the index
        
        ## ===============================================================        
        ## create the index for the current page and store it in current_page_index (current_page_index)
        ## current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}
        
        ## Example: if the curr_doc has id 1 and his text is 
        ##"web retrieval information retrieval":

        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}

        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        current_page_index = {}

        for position, term in enumerate(terms): # Loop over all terms
            try:
                # if the term is already in the index for the current page (current_page_index)
                # append the position to the corresponding list
        ## START CODE
                current_page_index[term][tweetId].append(position)  
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_page_index[term]=[tweetId, array('I',[position])] #'I' indicates unsigned int (int in Python)
            
        #merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)
        
        ## END CODE                    
                    
    return index


In [50]:
start_time = time.time()
index = create_index(lines)
print("Total time to create the index: {} seconds".format(np.round(time.time() - start_time, 2)))

print("Index results for the term 'researcher': {}\n".format(index['researcher']))
print("First 10 Index results for the term 'research': \n{}".format(index['intern'][:10]))

Total time to create the index: 1.71 seconds
Index results for the term 'researcher': []

First 10 Index results for the term 'research': 
[['0', array('I', [2])], ['2', array('I', [0])], ['35', array('I', [1])], ['262', array('I', [0])], ['273', array('I', [0])], ['277', array('I', [0])], ['278', array('I', [1])], ['280', array('I', [1])], ['299', array('I', [0])], ['338', array('I', [0])]]


In [ ]:
# 2. Make a proposal of 5 queries that will be used to evaluate your search engine
def search(query, index):
    """
    The output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query = build_terms(query)
    docs = set()
    for term in query:
    ## START DODE
        try:
            # store in term_docs the ids of the docs that contain "term"                        
            term_docs=["""YOUR CODE HERE""" for posting in index["""YOUR CODE HERE"""]]
            # docs = docs Union term_docs
            docs = """YOUR CODE HERE"""
        except:
            #term is not in index
            pass
    docs = list(docs)
    return docs

print("Insert your query (i.e.: Computer Science):\n")
query = input()
docs = search(query, index)
top = 10

print("\n======================\nSample of {} results out of {} for the searched query:\n".format(top, len(docs)))
for d_id in docs[:top]:
    print("page_id= {} - page_title: {}".format(d_id, title_index[d_id]))

In [ ]:
# 3. Apply a TF-IDF ranking to your results.
def create_index_tfidf(lines, num_documents):
    """
    Implement the inverted index and compute tf, df and idf
    
    Argument:
    lines -- collection of Wikipedia articles
    num_documents -- total number of documents
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of document these keys appears in (and the positions) as values.
    tf - normalized term frequency for each term in each document
    df - number of documents each term appear in
    idf - inverse document frequency of each term
    """

    index = defaultdict(list)
    tf = defaultdict(list)  #term frequencies of terms in documents (documents in the same order as in the main index)
    df = defaultdict(int)  #document frequencies of terms in the corpus
    title_index = defaultdict(str)
    idf = defaultdict(float)

    for line in lines:
        line_arr = line.split("|")
        page_id = int(line_arr[0])
        terms = build_terms(''.join(line_arr[1:]))  #page_title + page_text
        title = line_arr[1]
        title_index[page_id] = title

        ## ===============================================================        
        ## create the index for the **current page** and store it in current_page_index
        ## current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}

        ## Example: if the curr_doc has id 1 and his text is 
        ##"web retrieval information retrieval":

        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}

        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        current_page_index = {}

        for position, term in enumerate(terms):  ## terms contains page_title + page_text
            try:
                # if the term is already in the dict append the position to the corresponding list
                current_page_index["""YOUR CODE HERE"""]["""YOUR CODE HERE"""].append("""YOUR CODE HERE""")
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_page_index["""YOUR CODE HERE"""]=["""YOUR CODE HERE""", array('I',["""YOUR CODE HERE"""])] #'I' indicates unsigned int (int in Python)

        #normalize term frequencies
        # Compute the denominator to normalize term frequencies (formula 2 above)
        # norm is the same for all terms of a document.
        norm = 0
        for term, posting in current_page_index.items():
            # posting will contain the list of positions for current term in current document. 
            # posting ==> [current_doc, [list of positions]] 
            # you can use it to infer the frequency of current term.
            norm += len("""YOUR CODE HERE""") ** 2
        norm = math.sqrt(norm)

        #calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in current_page_index.items():
            # append the tf for current term (tf = term frequency in current doc/norm)
            tf[term].append(np.round(len("""YOUR CODE HERE""")/"""YOUR CODE HERE""",4)) ## SEE formula (1) above
            #increment the document frequency of current term (number of documents containing the current term)
            df[term] = """YOUR CODE HERE""" # increment DF for current term

        #merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)

        # Compute IDF following the formula (3) above. HINT: use np.log
        for term in df:
            idf[term] = np.round("""YOUR CODE HERE"""(float("""YOUR CODE HERE"""/"""YOUR CODE HERE""")), 4)

    return index, tf, df, idf, title_index


In [ ]:
start_time = time.time()
num_documents = len(lines)
index, tf, df, idf, title_index = create_index_tfidf(lines, num_documents)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time, 2)))

EVALUATION

In [ ]:

def rank_documents(terms, docs, index, idf, tf, title_index):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    title_index -- mapping between page id and page title
    
    Returns:
    Print the list of ranked documents
    """

    # I'm interested only on the element of the docVector corresponding to the query terms 
    # The remaining elements would became 0 when multiplied to the query_vector
    doc_vectors = defaultdict(lambda: [0] * len(terms)) # I call doc_vectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary
    query_vector = [0] * len(terms)

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms)  # get the frequency of each term in the query. 
    # Example: collections.Counter(["hello","hello","world"]) --> Counter({'hello': 2, 'world': 1})
    #HINT: use when computing tf for query_vector

    query_norm = la.norm(list(query_terms_count.values()))

    for termIndex, term in enumerate(terms):  #termIndex is the index of the term in the query
        if term not in index:
            continue

        ## Compute tf*idf(normalize TF as done with documents)
        query_vector[termIndex]="""YOUR CODE HERE"""/"""YOUR CODE HERE""" * """YOUR CODE HERE""" 

        # Generate doc_vectors for matching docs
        for doc_index, (doc, postings) in enumerate(index[term]):
            # Example of [doc_index, (doc, postings)]
            # 0 (26, array('I', [1, 4, 12, 15, 22, 28, 32, 43, 51, 68, 333, 337]))
            # 1 (33, array('I', [26, 33, 57, 71, 87, 104, 109]))
            # term is in doc 26 in positions 1,4, .....
            # term is in doc 33 in positions 26,33, .....

            #tf[term][0] will contain the tf of the term "term" in the doc 26            
            if doc in docs:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]  # TODO: check if multiply for idf

    # Calculate the score of each doc 
    # compute the cosine similarity between queyVector and each docVector:
    # HINT: you can use the dot product because in case of normalized vectors it corresponds to the cosine similarity
    # see np.dot
    
    doc_scores=[["""YOUR CODE HERE"""(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items() ]
    doc_scores.sort(reverse=True)
    result_docs = [x[1] for x in doc_scores]
    #print document titles instead if document id's
    #result_docs=[ title_index[x] for x in result_docs ]
    if len(result_docs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)
    #print ('\n'.join(result_docs), '\n')
    return result_docs


In [ ]:

def search_tf_idf(query, index):
    """
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query = build_terms(query)
    docs = set()
    for term in query:
        try:
            # store in term_docs the ids of the docs that contain "term"                        
            term_docs=["""YOUR CODE HERE""" for posting in index["""YOUR CODE HERE"""]]
            
            # docs = docs Union term_docs
            docs = """YOUR CODE HERE"""
        except:
            #term is not in index
            pass
    docs = list(docs)
    ranked_docs = rank_documents(query, docs, index, idf, tf, title_index)
    return ranked_docs

print("Insert your query (i.e.: Computer Science):\n")
query = input()
ranked_docs = search_tf_idf(query, index)
top = 10

print("\n======================\nTop {} results out of {} for the searched query:\n".format(top, len(ranked_docs)))
for d_id in ranked_docs[:top]:
    print("page_id= {} - page_title: {}".format(d_id, title_index[d_id]))